### Step 1 - Import Archives and Librarys

In [12]:
import pandas as pd
import pathlib as pl
import win32com.client as win32
import pythoncom

In [13]:
emails = pd.read_excel(r'Databases/emails.xlsx')
stores = pd.read_csv(r'Databases/stores.csv', encoding='latin-1', sep=';')   # Using latin-1 encoding because the file has some characteres not recognized by utf-8
sales = pd.read_excel(r'Databases/sales.xlsx')

### Step 2 - Define one sheet to each store and definne the indicator day


In [14]:
# include store name to "sales"

sales = sales.merge(stores, on='Store ID')

In [15]:
store_dictionary = {}

for item in stores['Store']:
    store_dictionary[item] = sales.loc[sales['Store']==item, :]

In [16]:
# Find the latest day available and get its data

indicator_day = sales['Date'].max()

### Step 3 - Save the sheets in the Backup folder


In [17]:
# identify if folder already exists
backup_path = pl.Path(r'Backup')
backup_folder_files = backup_path.iterdir()
backup_name_list = []

for file in backup_folder_files:
    backup_name_list.append(file.name)

In [18]:
# save inside the respective folder
for store in store_dictionary:
    if store not in backup_name_list:
        new_folder = backup_path / store
        new_folder.mkdir()

    file_name = '{}_{}_{}.xlsx'.format(indicator_day.month, indicator_day.day, store)

    file_path = backup_path / store / file_name

    store_dictionary[store].to_excel(file_path)


### Step 4 - Calculate the indicators (Revenue from the year, revenue from the last day in dataframe, product diversity from sales and average ticket from each store)


In [19]:
for store in store_dictionary:
    store_sales_year = store_dictionary[store]
    store_sales_day = store_sales_year.loc[store_sales_year['Date']==indicator_day, :]

    # Revenue indicators
    revenue_year = store_sales_year['Final Value'].sum()
    revenue_day = store_sales_day['Final Value'].sum()

    # Product diversity indicators
    product_amount_year = len(store_sales_year['Product'].unique())
    product_amount_day = len(store_sales_day['Product'].unique())

    # Average ticket indicators
    sale_values_year = store_sales_year.groupby('Sale Code').sum('Final Value')
    sale_values_day = store_sales_day.groupby('Sale Code').sum('Final Value')
    average_ticket_year = sale_values_year['Final Value'].mean()
    average_ticket_day = sale_values_day['Final Value'].mean()

In [20]:
# Define goals
revenue_year_goal = 1000
revenue_day_goal = 1650000
product_amount_year_goal = 4
product_amount_day_goal = 120
average_ticket_year_goal = 500
average_ticket_day_goal = 500


### Step 5 - Send email to the store manager


In [21]:
attachment = pl.Path.cwd() / backup_path / store / '{}_{}_{}.xlsx'.format(indicator_day.month, indicator_day.day, store)

print(attachment)

c:\Users\Pichau\Documents\GitHub\Store-Indicators-Automation\Backup\Shopping Barra\12_26_Shopping Barra.xlsx


In [23]:
outlook = win32.Dispatch('outlook.application', pythoncom.CoInitialize())

name = emails.loc[emails['Store']==store, 'Manager'].values[0]
mail = outlook.CreateItem(0)
mail.To = emails.loc[emails['Store']==store, 'Email'].values[0]
mail.Subject = 'Indicators - Date {}/{} ({})'.format(indicator_day.month, indicator_day.day, store)
# Use mail.Body if you want to make a simple text mail.
mail.HTMLBody = '''
<p>Hi {},</p>

<p>Attached is the financial report for the <strong>{}</strong> store from <strong>yesterday ({}/{})</strong>.</p>

<p>&nbsp;</p>

<p>Here is a brief summary:</p>

<table align="center" border="1" cellpadding="1" cellspacing="1" style="width:500px">
	<thead>
		<tr>
			<th scope="col">Indicator</th>
			<th scope="col">Day Value</th>
			<th scope="col">Day Goal</th>
			<th scope="col">Day Scenario</th>
		</tr>
	</thead>
	<tbody>
		<tr>
			<td>
			<p>Revenue</p>
			</td>
			<td style="text-align: center;">{}</td>
			<td style="text-align: center;">{}</td>
			<td style="text-align: center;">◙</td>
		</tr>
		<tr>
			<td>
			<p>Product Diversity</p>
			</td>
			<td style="text-align: center;">{}</td>
			<td style="text-align: center;">{}</td>
			<td style="text-align: center;">◙</td>
		</tr>
		<tr>
			<td>
			<p>Average Ticket</p>
			</td>
			<td style="text-align: center;">{}</td>
			<td style="text-align: center;">{}</td>
			<td style="text-align: center;">◙</td>
		</tr>
	</tbody>
</table>

<p>&nbsp;</p>

<hr />
<p style="text-align: center;"><strong>Please note that this is an automated email. For any questions, please contact the headquarter.</strong></p>

<p style="text-align: center;">&nbsp;</p>

<p style="text-align: center;">Best regards,</p>

<p style="text-align: center;">SIA - Store Indicators Automation.</p>
'''

attachment = pl.Path.cwd() / backup_path / store / '{}_{}_{}.xlsx'.format(indicator_day.month, indicator_day.day, store)
mail.Attachments.Add(str(attachment))

mail.Send()

### Step 6 - Automate every store


### Step 7 - Create ranking for the director of the company


### Step 8 - Send email to the director